In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import torch
import random  # 确保导入了random
import pandas as pd
import numpy as np
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import TensorDataset, DataLoader
import openpyxl
from pandas.io.excel import ExcelWriter

%matplotlib inline

In [ ]:
no_cuda= False
use_cuda = True
batch_size = 32
latent_size = 20  # z dim
cuda_available = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")


kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST("data", train=True, download=True, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST("data", train=False, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
)

In [ ]:
def to_var(x):
    x = Variable(x)
    # if use_cuda:
    #     x = x.cuda()
    return x


def one_hot(labels, class_size):
    targets = torch.zeros(labels.size(0), class_size)
    for i, label in enumerate(labels):
        targets[i, label] = 1
    return to_var(targets)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(len(x),-1))
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    beta = 0.5 # the weight of KLD in the loss
    return BCE + beta*KLD


def KLD(mu, logvar):
    # BCE = F.binary_cross_entropy(recon_x, x.view(len(x),-1))
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return  KLD

In [ ]:
class CVAE(nn.Module):
    def __init__(self, feature_size, latent_size, class_size):
        super(CVAE, self).__init__()
        self.feature_size = feature_size
        self.class_size = class_size

        # encode
        self.fc1 = nn.Linear(feature_size + class_size, 1000)
        self.fc21 = nn.Linear(1000, latent_size)
        self.fc22 = nn.Linear(1000, latent_size)

        # decode
        self.fc3 = nn.Linear(latent_size + class_size, 1000)
        self.fc4 = nn.Linear(1000, feature_size)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x, c):  # Q(z|x, c)
        """
        x: (bs, feature_size)
        c: (bs, class_size)
        """
        inputs = torch.cat([x, c], 1)  # (bs, feature_size+class_size)
        h1 = self.relu(self.fc1(inputs))
        z_mu = self.fc21(h1)
        z_var = self.fc22(h1)
        return z_mu, z_var

    def reparametrize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std) + mu
        else:
            return mu

    def decode(self, z, c):  # P(x|z, c)
        """
        z: (bs, latent_size)
        c: (bs, class_size)
        """
        inputs = torch.cat([z, c], 1)  # (bs, latent_size+class_size)
        h3 = self.relu(self.fc3(inputs))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x, c):
        mu, logvar = self.encode(x, c)
        z = self.reparametrize(mu, logvar)
        return self.decode(z, c), mu, logvar

In [ ]:
'''def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, labels) in enumerate(train_loader):
        data = to_var(data).view(len(data), -1)
        labels = one_hot(labels, 2)
        recon_batch, mu, logvar = model(data, labels)
        optimizer.zero_grad()
        loss = KLD( mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 5 == 0: #这里的5是指每5个print一次结果
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item() / len(data),
                )
            )'''
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, labels) in enumerate(train_loader):
        data = to_var(data).view(len(data), -1)
        labels = one_hot(labels, 2)
        recon_batch, mu, logvar = model(data, labels)
        optimizer.zero_grad()
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 5 == 0: #这里的5是指每5个print一次结果
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item() / len(data),
                )
            )

In [ ]:
#input features
data = pd.read_excel("D:\CVAE model\Fe-Co-Ni-Ti-Zr\Dataset_18PI.xlsx", header=None)
label=pd.read_excel("D:\CVAE model\Fe-Co-Ni-Ti-Zr\Labels.xlsx")

In [ ]:
X = torch.from_numpy(data.values).float().T
y=torch.from_numpy(label.values)
#y=torch.nn.functional.one_hot(y,3).float()

#label=torch.distributions.OneHotCategorical(torch.tensor([0.3,0.5,0.2])).sample([100])
#y=torch.distributions.Categorical(torch.tensor([0.3,0.5,0.2])).sample([100])
dat_X = TensorDataset(X,y)
train_loader = DataLoader(dat_X,batch_size=10)
print(dat_X)

In [ ]:
model = CVAE(23, 2, 2)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(1, 5):
    train(epoch)

In [ ]:
z=model.encode(X,one_hot(y,2))[0].detach().numpy() #这里的[0]是指均值 [1]是指方差
plt.scatter(z[:,0],z[:,1],c=y.numpy())

In [ ]:
print(z)
df = pd.DataFrame(z)
with pd.ExcelWriter("D:\CVAE model\Fe-Co-Ni-Ti-Zr\Gene_vector.xlsx", mode="a", if_sheet_exists='replace', engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="latent_space")

In [ ]:
for epoch in range(1, 2):
    train(epoch)
    #test(epoch)
    with torch.no_grad():
        #采样过程
        sample = torch.randn(1000, 2).to(device)
        #print(sample)
        c = np.zeros(shape=(sample.shape[0],))
        #print(c)
        rand = np.random.randint(0, 1)
        #print(f"Random number: {rand}")
        c[:] = rand
        c = torch.IntTensor(c)
        c = one_hot(c,2)
        #print(c)
        #print(sample.shape)
        sample = model.decode(sample, c)
        #模型的输出矩阵：每一行的末尾都加了one-hot向量，要去掉这个one-hot向量再转换为图片。
        #generated_image = sample[:, 0:sample.shape[1]-10]
        #print(sample)
        b=sample.numpy().tolist()
        #print(b)
        #sample.shape
        print(b)

        #下面两个语句可以把tensor写进csv里
        #b = sample.numpy()
        #np.savetxt("ZrFeTi34composition.csv",b)

In [ ]:
#把数据添加到一个excel
df = pd.DataFrame(b)
with pd.ExcelWriter("D:\CVAE model\Fe-Co-Ni-Ti-Zr\Gene_vector.xlsx", mode="a", if_sheet_exists='replace', engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="2")

In [ ]:
z=model.encode(X,one_hot(y,2))[0].detach().numpy()
c = np.zeros(shape=(sample.shape[0],))
print(c)
rand = np.random.randint(0, 1)
print(f"Random number: {rand}")
c[:] = rand
c = torch.IntTensor(c)
c = one_hot(c,2)
print(c)
sample = model.decode(z, c)

In [ ]:
z0=torch.tensor([0.05,0.2]).view(1,2)
y0=torch.tensor([[0]])
y0=one_hot(y0,2)
rec_x=model.decode(z0,y0)
print(rec_x)